# 🎓 The Virtual NMR Spectrometer

Welcome to the **Interactive Relaxation Lab**! 

This notebook simulates a **Nuclear Magnetic Resonance (NMR)** experiment on a virtual protein.
NMR relaxation rates ($R_1$, $R_2$, and Heteronuclear NOE) are the primary tools biophysicists use to study protein movement at the atomic level.

### 🎯 Goal
Understand how **molecular size** (tumbling rate) and **magnetic field strength** affect the signals we measure. This simulation uses the standard *Lipari-Szabo Model-Free* formalism, which is the gold standard for analyzing protein dynamics.

In [ ]:
# ⚙️ Install Dependencies
# We install openmm via pip, which is compatible with Colab default environment.
try:
    import google.colab
    print("Installing OpenMM and synth-pdb via Pip...")
    !pip install -q openmm synth-pdb matplotlib ipywidgets py3Dmol
except ImportError:
    print("Assuming local environment.")
    # If running locally, please ensure dependencies are installed via pip or conda.
    pass


In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
import numpy as np
from synth_pdb.generator import generate_pdb_content
from synth_pdb.relaxation import calculate_relaxation_rates, predict_order_parameters
import biotite.structure.io.pdb as pdb
import io

# 1. Generate a Test Protein (Helix-Turn-Helix)
# distinct regions: Rigid Helices vs Flexible Loop
sequence = "AAAAAAGGGAAAAAA"
structure_def = "1-6:alpha,7-9:random,10-15:alpha"

print("🧬 Generating virtual protein...")
pdb_content = generate_pdb_content(sequence_str=sequence, structure=structure_def)
pdb_file = pdb.PDBFile.read(io.StringIO(pdb_content))
structure = pdb_file.get_structure(model=1)

# Pre-calculate Order Parameters (S2) based on structure
s2_map = predict_order_parameters(structure)
res_ids = sorted(list(s2_map.keys()))
s2_values = [s2_map[r] for r in res_ids]

print("✅ Protein Model Ready!")

### 🧬 The Simulated Protein

We have generated a synthetic **Helix-Turn-Helix** motif to demonstrate contrast between rigid and flexible regions:
*   **Residues 1-6**: Alpha Helix (Rigid Structure)
*   **Residues 7-9**: Flexible Loop (Disordered)
*   **Residues 10-15**: Alpha Helix (Rigid Structure)

Notice how the physics engine (`synth-pdb`) automatically assigns different **Order Parameters ($S^2$)** to these regions. $S^2$ represents spatial restriction: **1.0** is completely rigid, **0.0** is completely disordered.

In [ ]:
# Visualizing the Structure
import py3Dmol

view = py3Dmol.view(width=400, height=300)
view.addModel(pdb_content, 'pdb')
view.setStyle({'cartoon': {'color': 'spectrum'}})
view.zoomTo()
print("👀 3D View of Helix-Turn-Helix (Blue=N-term, Red=C-term)")
view.show()

### 📊 Guide to the Plots

When you run the simulator below, you will see three coupled plots. Here is how to interpret them:

1.  **$R_1$ (Longitudinal Rate - Blue)**: 
    *   *Physics*: Sensitive to fast interaction fluctuations (nanosecond scale).
    *   *Pattern*: Often relatively flat for folded proteins, but may dip in flexible loops.

2.  **$R_2$ (Transverse Rate - Red)**: 
    *   *Physics*: Sensitive to slow motions (global tumbling) and chemical exchange.
    *   *Key Insight*: **$R_2$ scales with protein size**. Large proteins (slow tumbling) have high $R_2$ rates. High $R_2$ means the signal decays fast (broad lines), making large proteins hard to study!
    *   *Flexible Regions*: $R_2$ drops sharply because local flexibility averages out the magnetic interactions.

3.  **Heteronuclear NOE (Green)**: 
    *   *The Rigidity Sensor*: This is the most robust indicator of local structure.
    *   **Value ~ 0.8**: Rigid backbone (Helix/Sheet).
    *   **Value < 0.6**: Flexible loop/terminus.
    *   **Negative Values**: Extremely flexible or unfolded.

---

In [ ]:
def plot_relaxation(field_mhz=600, tau_m_ns=10.0):
    """Calculate and plot rates for given conditions."""
    
    # Calculate Rates using synth-pdb physics engine
    rates = calculate_relaxation_rates(
        structure, 
        field_mhz=field_mhz, 
        tau_m_ns=tau_m_ns, 
        s2_map=s2_map
    )
    
    r1 = [rates[r]['R1'] for r in res_ids]
    r2 = [rates[r]['R2'] for r in res_ids]
    noe = [rates[r]['NOE'] for r in res_ids]
    
    # Plotting
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 12), sharex=True)
    
    # R1 Plot
    ax1.plot(res_ids, r1, 'o-', color='blue', label='R1 (Longitude)')
    ax1.set_ylabel('$R_1$ ($s^{-1}$)')
    ax1.set_title(f'Relaxation Dynamics @ {field_mhz} MHz, $\\tau_m$={tau_m_ns}ns')
    ax1.grid(True, alpha=0.3)
    
    # R2 Plot
    ax2.plot(res_ids, r2, 's-', color='red', label='R2 (Transverse)')
    ax2.set_ylabel('$R_2$ ($s^{-1}$)')
    ax2.grid(True, alpha=0.3)
    
    # NOE Plot
    ax3.plot(res_ids, noe, '^-', color='green', label='HetNOE')
    ax3.set_ylabel('NOE Ratio')
    ax3.set_xlabel('Residue Number')
    ax3.grid(True, alpha=0.3)
    
    # Highlight the flexible loop region (7-9)
    for ax in [ax1, ax2, ax3]:
        ax.axvspan(7, 9, color='yellow', alpha=0.2, label='Flexible Loop')
        ax.legend()
        
    plt.show()

# Create Interactive Widgets
print("🎛 Starting Virtual Spectrometer...")
interact(
    plot_relaxation, 
    field_mhz=widgets.IntSlider(min=400, max=1200, step=100, value=600, description='Field (MHz)'),
    tau_m_ns=widgets.FloatSlider(min=2.0, max=50.0, step=1.0, value=10.0, description='Tumbling (ns)')
);

### 🧪 Try These Experiments

Use the sliders above to change the "experimental" conditions:

1.  **Simulate a Giant Protein Complex**:
    *   Slide **Tumbling (ns)** to **40.0 ns**.
    *   *Observation*: Look at the **$R_2$ (Red)** plot. It shoots up significantly! This is why NMR is limited to smaller proteins (< 50-100 kDa) without special tricks (like TROSY).
    *   *Note*: The NOE profile stays mostly the same—local rigidity hasn't changed, only the global tumbling.

2.  **Go to High Field**:
    *   Slide **Field (MHz)** from **600** to **1200**.
    *   *Observation*: $R_2$ increases. This is due to **Chemical Shift Anisotropy (CSA)** becoming a stronger relaxation mechanism at high magnetic fields.
